In [1]:
import streamlit as st

import requests
from bs4 import BeautifulSoup
from rotten_tomatoes_scraper.rt_scraper import MovieScraper
import imdb
import mal

In [2]:
user_agent = {'User-agent': 'Mozilla/5.0'}

# Streamlit Website

In [ ]:
st.set_page_config(page_title='Review aggregator', page_icon=':anger:')

#-----HEADER SECTION
st.subheader('Review Aggregator - Movies/Tv shows/Animes')
st.title('Title:')
title = st.text_input(label='Insert title')
genre = st.radio('Choose the genre', ('Movie', 'Tv Show', 'Anime'))


## Metacritic

In [24]:
def search_meta(title, genre):
    # Genre can be 'tv' or 'movie'
    if genre == 'Movie':
        genre = 'movie'
    else:
        genre = 'tv'
    
    title = title.lower().replace(' ', '-').replace(':', '')
    meta_url = 'https://www.metacritic.com/' + genre + '/' + title
    meta = requests.get(meta_url, headers=user_agent)
    soup = BeautifulSoup(meta.text, 'html.parser')
    meta_critic_rev = float(soup.find('td', class_='summary_right').text) / 10
    meta_user_rev = soup.find('td', class_='summary_right pad_btm1').text
    if meta_user_rev != '\n\ntbd\n\n':
        meta_user_rev = float(meta_user_rev)
    else:
        meta_user_rev = 'No score yet'
    
    print(meta_url)
    print('Critics score:', meta_critic_rev)
    print('Users score:', meta_user_rev)

In [25]:
search_meta('Howls moving castle', 'Movie')

'https://www.metacritic.com/movie/howls-moving-castle'

## Rotten Tomatoes

In [28]:
def search_rt(title, genre):
    # Genre 'tv' or 'm'
    # Genre can be 'tv' or 'movie'
    if genre == 'Movie':
        genre = 'm'
    else:
        genre = 'tv'
    
    title = title.lower().replace(' ', '_').replace(':', '')
    rt_url = 'https://www.rottentomatoes.com/' + genre + '/' + title
    rt = requests.get(rt_url, headers=user_agent)
    soup = BeautifulSoup(rt.text, 'html.parser')
    if genre == 'tv':
        reviews = soup.findAll('span', class_='mop-ratings-wrap__percentage')
        rt_critic_rev = int(reviews[0].text.replace(' ', '').replace('\n', '').replace('%', '')) / 10
        rt_user_rev = int(reviews[1].text.replace(' ', '').replace('\n', '').replace('%', '')) / 10   
    else:
        movie_scraper = MovieScraper(movie_url=rt_url)
        movie_scraper.extract_metadata()
        reviews = movie_scraper.metadata
        rt_critic_rev = int(reviews['Score_Rotten']) / 10
        rt_user_rev = int(reviews['Score_Audience']) / 10
    
    print(rt_url)
    print('Critics score:', rt_critic_rev)
    print('Users score:', rt_user_rev)

In [94]:
search_rt('tatami', 'tv')

IndexError: list index out of range

## IMDb

In [84]:
def search_imdb(title):
    ia = imdb.Cinemagoer()
    try:
        # Do the search, and get the results (a list of Movie objects).
        results = ia.search_movie(title)
    except imdb.IMDbError as e:
        print("Probably you're not connected to Internet.  Complete error report:")
        print(e)
        sys.exit(3)
    movie = ia.search_movie(title.lower())
    print(movie)
    imdb_id = movie[0].getID()
    imdb_url = 'https://imdb.com/title/tt' + imdb_id
    im = requests.get(imdb_url, headers=user_agent)
    soup = BeautifulSoup(im.text, 'html.parser')
    score = float(soup.find('span', class_='sc-7ab21ed2-1 jGRxWM').text)
    print(imdb_url)
    print(score)

In [85]:
search_imdb('the office')

[<Movie id:0386676[http] title:_"The Office" (2005)_>, <Movie id:0290978[http] title:_"The Office" (2001)_>, <Movie id:20877972[http] title:_"The Office" (2022)_>, <Movie id:14819828[http] title:_"The Office Blind Date" (2022)_>, <Movie id:9114660[http] title:_The Office Mix-Up (2020)_>, <Movie id:8305218[http] title:_"The Office" (2019)_>, <Movie id:7921562[http] title:_The Office Games (2022)_>, <Movie id:0840149[http] title:_"The Office: The Accountants" (2006) (mini)_>, <Movie id:0151804[http] title:_Office Space (1999)_>, <Movie id:0112108[http] title:_"The Office" (1995)_>, <Movie id:1791001[http] title:_"The Office" (2010)_>, <Movie id:0139786[http] title:_The Office (1996) (TV)_>, <Movie id:15019916[http] title:_"The Office PL" (2021)_>, <Movie id:6097546[http] title:_The Office (1981)_>, <Movie id:0167319[http] title:_The Office (1966)_>, <Movie id:12194000[http] title:_Out of Office (2022) (TV)_>, <Movie id:1740835[http] title:_"The Office: The 3rd Floor" (2010)_>, <Movie id:

## My anime list

In [72]:
def search_mal(title):
    search = mal.AnimeSearch(title)
    return search.results[0].url, search.results[0].score